In [1]:
%reload_ext autoreload
%autoreload 2
#%reload_ext notexbook
#%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
%matplotlib qt5

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import torch

import os

os.chdir('/home/axtr7550/Chromosome_project/cellbgnet/')
import cellbgnet
import cellbgnet.utils

from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.simulation.psf_kernel import SMAPSplineCoefficient
from cellbgnet.generic.emitter import EmitterSet
from cellbgnet.train_loss_infer import generate_probmap_cells

os.chdir('/home/axtr7550/Chromosome_project/cellbgnet/notebooks')

from skimage.io import imread
from skimage.measure import label
import random
import edt
from skimage.filters import gaussian
from scipy.ndimage import rotate
import random
import pickle
import pathlib
from pathlib import Path
%matplotlib qt5

In [5]:
param_file = '../cellbgnet/utils/reference_files/reference_Axel.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [6]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
train_size = simulation_params['train_size']

### Testing the model

In [7]:
model = cellbgnet.model.CellBGModel(param)

INITIATING CUDA IMPLEMENTATION
training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21,

In [8]:
model.training(128, simulation_params)

/home/axtr7550/anaconda3/envs/compile_spline/lib/python3.11/site-packages/imageio/plugins/pillow.py:297: UserWarning: Loading 16-bit (uint16) PNG as int32 due to limitations in pillow's PNG decoder. This will be fixed in a future version of pillow which will make this warning dissapear.
  warnings.warn(


(tensor(202462.7812, device='cuda:0'),
 (tensor(30.6710, device='cuda:0'),
  tensor(88.8533, device='cuda:0'),
  tensor(2529.3955, device='cuda:0'),
  tensor(64.6259, device='cuda:0')))

### Testing data simulator

In [9]:
data_gen = DataSimulator(psf_params, simulation_params, hardware_params)

INITIATING CUDA IMPLEMENTATION


In [10]:
prob_map = np.zeros([1, train_size, train_size])
prob_map[0, int(simulation_params['margin_empty'] * train_size): int((1 - simulation_params['margin_empty']) * train_size),
         int(simulation_params['margin_empty'] * train_size): int((1 - simulation_params['margin_empty']) * train_size)] += 1
prob_map = prob_map / prob_map.sum() * 4

In [11]:
imgs_sim, xyzi_gt, s_mask, psf_imgs_gt, locs = data_gen.simulate_data(gpu(prob_map), batch_size=128)

type: 'DataSimulator' object has no attribute 'sliding_win'

In [12]:
data_gen.device

'cuda:0'

In [13]:
imgs_sim.shape

torch.Size([128, 1, 128, 128])

In [14]:
plt.figure()
plt.imshow(imgs_sim[3, 0].cpu().numpy())
plt.show()

In [15]:
plt.figure()
plt.imshow(psf_imgs_gt[3].cpu())

In [17]:
imgs_sim.shape

torch.Size([128, 1, 128, 128])

In [18]:
psf_imgs_gt.shape

torch.Size([128, 128, 128])

In [19]:
xyzi_gt.shape

torch.Size([128, 10, 4])

In [16]:
s_mask[3,:]

tensor([1., 1., 1., 1., 1., 0., 0., 0., 0., 0.], device='cuda:0')

In [17]:
xyzi_gt[3, :]

tensor([[ 81.7735,  16.0448,   0.6458,   0.9556],
        [  8.2155,  46.8738,  -0.2595,   0.5951],
        [117.7596,  64.0497,  -0.4851,   0.6828],
        [ 54.7972,  79.3062,   0.5025,   0.9386],
        [ 97.5091, 100.9258,   0.9972,   0.6686],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000]], device='cuda:0')